# 🚀 Simple Pipeline - Interactive Demo

Este notebook muestra cómo usar Simple Pipeline para crear datasets sintéticos con Ollama.

## 📋 Contenido
1. Setup e imports
2. Pipeline básico
3. Pipeline con filtros y transformaciones
4. Pipeline robusto con manejo de errores
5. Análisis de resultados

## 1. Setup e Imports

In [ ]:
import pandas as pd
import sys
from pathlib import Path

# Agregar el directorio padre al path para imports
sys.path.insert(0, str(Path.cwd().parent))

from simple_pipeline import SimplePipeline
from simple_pipeline.steps import (
    LoadDataFrame,
    OllamaLLMStep,
    RobustOllamaStep,
    FilterRows,
    SortRows,
    SampleRows,
    AddColumn,
    KeepColumns
)

print("✅ Imports completados")

## 2. Pipeline Básico

Empezamos con un pipeline simple que genera explicaciones de conceptos.

In [ ]:
# Datos de entrada
concepts = pd.DataFrame({
    'concept': ['Machine Learning', 'Blockchain', 'Quantum Computing'],
    'audience': ['beginner', 'intermediate', 'advanced']
})

print("📊 Datos de entrada:")
display(concepts)

In [ ]:
# Crear pipeline
pipeline = SimplePipeline(
    name="basic-demo",
    description="Demo básico de explicaciones"
)

# Agregar steps
pipeline.add_step(
    LoadDataFrame(name="load", df=concepts)
)

pipeline.add_step(
    OllamaLLMStep(
        name="explain",
        model_name="llama3.2",
        prompt_column="concept",
        output_column="explanation",
        prompt_template=lambda row: f"Explain {row['concept']} to a {row['audience']} audience in 2-3 sentences.",
        system_prompt="You are a clear, concise technical educator.",
        batch_size=3,
        generation_kwargs={"temperature": 0.7, "num_predict": 100}
    )
)

print("✅ Pipeline configurado")

In [ ]:
# Ejecutar pipeline
result = pipeline.run(use_cache=True)

print("\n📊 Resultados:")
display(result)

## 3. Pipeline con Filtros y Transformaciones

Ahora vamos a crear un pipeline más complejo con varios steps de transformación.

In [ ]:
# Dataset más grande
topics_data = pd.DataFrame({
    'topic': [
        'Python', 'JavaScript', 'Rust', 'Go', 'Java',
        'C++', 'Ruby', 'Swift', 'Kotlin', 'TypeScript'
    ],
    'type': [
        'interpreted', 'interpreted', 'compiled', 'compiled', 'compiled',
        'compiled', 'interpreted', 'compiled', 'compiled', 'interpreted'
    ],
    'popularity': [95, 85, 75, 80, 90, 70, 60, 65, 70, 85]
})

print("📊 Dataset inicial:")
display(topics_data)

In [ ]:
# Pipeline con transformaciones
transform_pipeline = SimplePipeline(
    name="transform-demo",
    description="Demo con filtros y transformaciones"
)

# Step 1: Cargar datos
transform_pipeline.add_step(
    LoadDataFrame(name="load", df=topics_data)
)

# Step 2: Filtrar solo lenguajes populares (> 75)
transform_pipeline.add_step(
    FilterRows(
        name="filter_popular",
        filter_column="popularity",
        condition="> 75"
    )
)

# Step 3: Ordenar por popularidad
transform_pipeline.add_step(
    SortRows(
        name="sort",
        by="popularity",
        ascending=False
    )
)

# Step 4: Tomar top 3
transform_pipeline.add_step(
    SampleRows(
        name="top_3",
        n=3
    )
)

# Step 5: Generar características
transform_pipeline.add_step(
    OllamaLLMStep(
        name="generate_features",
        model_name="llama3.2",
        prompt_column="topic",
        output_column="key_features",
        prompt_template=lambda row: f"List 3 key features of {row['topic']} programming language. Be concise.",
        batch_size=3
    )
)

print("✅ Pipeline de transformación configurado")

In [ ]:
# Ejecutar
result_transform = transform_pipeline.run(use_cache=True)

print("\n📊 Top 3 lenguajes más populares con features:")
display(result_transform)

## 4. Pipeline Robusto con Manejo de Errores

Usamos `RobustOllamaStep` para manejar errores de forma elegante.

In [ ]:
# Dataset que puede causar problemas
tricky_data = pd.DataFrame({
    'task': [
        'Write a function to sort a list',
        'Explain recursion',
        'This is an intentionally bad prompt ###@@@',  # Puede fallar
        'Implement binary search',
    ]
})

display(tricky_data)

In [ ]:
# Pipeline robusto
robust_pipeline = SimplePipeline(
    name="robust-demo",
    description="Demo de manejo robusto de errores"
)

robust_pipeline.add_step(
    LoadDataFrame(name="load", df=tricky_data)
)

robust_pipeline.add_step(
    RobustOllamaStep(
        name="generate_code",
        model_name="llama3.2",
        prompt_column="task",
        output_column="code",
        system_prompt="You are a coding assistant. Generate Python code.",
        batch_size=2,
        max_retries=2,
        save_failures=True,
        continue_on_error=True  # Continuar aunque haya errores
    )
)

print("✅ Pipeline robusto configurado")

In [ ]:
# Ejecutar
result_robust = robust_pipeline.run(use_cache=True)

print("\n📊 Resultados (incluyendo errores):")
display(result_robust[['task', 'status', 'error']])

In [ ]:
# Ver solo resultados exitosos
successful = result_robust[result_robust['status'] == 'success']
print(f"\n✅ {len(successful)} de {len(result_robust)} fueron exitosos")
display(successful[['task', 'code']])

## 5. Análisis de Resultados

Análisis más detallado de los resultados generados.

In [ ]:
# Estadísticas del pipeline básico
print("📊 Análisis del Pipeline Básico:")
print(f"Total de filas: {len(result)}")
print(f"Columnas: {list(result.columns)}")

# Longitud de explicaciones
result['explanation_length'] = result['explanation'].str.len()
print(f"\nLongitud promedio de explicaciones: {result['explanation_length'].mean():.0f} caracteres")

# Visualizar
import matplotlib.pyplot as plt

result['explanation_length'].plot(kind='bar', title='Longitud de Explicaciones')
plt.ylabel('Caracteres')
plt.xlabel('Concepto')
plt.xticks(range(len(result)), result['concept'], rotation=45)
plt.tight_layout()
plt.show()

## 🎯 Conclusión

Has aprendido a:
- ✅ Crear pipelines básicos con SimplePipeline
- ✅ Usar filtros y transformaciones
- ✅ Manejar errores robustamente
- ✅ Analizar resultados

### 📚 Próximos pasos:
1. Experimenta con diferentes modelos de Ollama
2. Crea tus propios steps personalizados
3. Prueba con datasets más grandes
4. Explora el caching para acelerar iteraciones

## 🧹 Limpieza

Opcional: limpiar cache de los pipelines

In [ ]:
# Descomentar para limpiar cache
# pipeline.clear_cache()
# transform_pipeline.clear_cache()
# robust_pipeline.clear_cache()
# print("✅ Cache limpiada")